### This Microsoft Fabric notebook exports all the tables in a Microsoft Fabric Semantic Model as tables in a Microsoft Fabric Lakehouse. It also lists all the measures, including their definition.

Author: Luis Soares<br>Contibutor: Andreas Bergstedt

Thanks to: https://learn.microsoft.com/en-us/fabric/data-science/read-write-power-bi-spark

By default, the workspace used to access semantic models is:
- the workspace of the attached Lakehouse or
- the workspace of the notebook, if no Lakehouse is attached.

In [ ]:
%pip install semantic-link
%load_ext sempy

In [ ]:
# change these 3 parameters acording to your environment

workspace_name        = "Microsoft Fabric Capacity Metrics" # workspace name where the semantic model is
semantic_model_name   = "Fabric Capacity Metrics"           # semantic model name
target_lakehouse_name = "Fabric_Capacity_Metrics_LH"        # target lakehouse name (must be added as source)

In [ ]:
import sempy.fabric as fabric
import pyspark.sql.functions as F
df_datasets = fabric.list_datasets(workspace_name) # datasets are semantic models
df_datasets

In [ ]:
# this is not needed for an export, but gives a visual relationship between tables

from sempy.relationships import plot_relationship_metadata

relationships = fabric.list_relationships(semantic_model_name, workspace = workspace_name)
plot_relationship_metadata(relationships)

In [ ]:
df_tables = fabric.list_tables(semantic_model_name, workspace = workspace_name) # list all tables from a semantic model
df_tables

In [ ]:
# use this cell to show a specific table. not needed to export data
df_test = fabric.read_table(semantic_model_name, "Dates", workspace = workspace_name)
df_test.head(10)

In [ ]:
total_tables = df_tables.shape[0]
lakehouse_path = "/" + target_lakehouse_name + "/Tables/"
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

for index, row in df_tables.iterrows():
    table_name = row["Name"]
    print(f"Exporting table {index + 1}/{total_tables}: {table_name}")

    if(table_name == "All Measures"): # this table can't be execute because it has no columns. will be handled after
        print("Skipping...")
        continue

    # https://learn.microsoft.com/en-us/fabric/data-engineering/lakehouse-notebook-load-data
    pd_table = fabric.read_table(semantic_model_name, table_name, workspace = workspace_name)

    n_rows = pd_table.shape[0]
    print("# rows:", n_rows)
    if(n_rows == 0):            # if a table is empty in the semantic model, we can't infer the column types
        print("Skipping...")
        continue

    df_table = spark.createDataFrame(pd_table)

    df_table_renamed_collumns = df_table
    for col in df_table.columns:
        df_table_renamed_collumns = df_table_renamed_collumns.withColumnRenamed(col, col.replace(" ", "_").replace("(", "_").replace(")", "_"))

    # write as a table in the lakehouse
    df_table_renamed_collumns.write.mode("overwrite").format("delta").saveAsTable(table_name.replace(" ", "_"))


In [ ]:
df_measures = fabric.list_measures(semantic_model_name, workspace = workspace_name)
display(df_measures)